In [1]:
# Standard imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
print('- Finish importing standard packages')

# setup selenium webdriver
opt = webdriver.ChromeOptions()
#opt.add_extension("Block-image_v1.1.crx")
opt.add_argument('--disable-gpu')
opt.add_argument("--window-size=1920,1080")
opt.add_argument("--start-maximized")
opt.add_argument('--disable-dev-shm-usage')
opt.add_argument('--no-sandbox')
opt.add_argument('--ignore-certificate-errors')
#opt.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36")
driver = webdriver.Chrome(options=opt)

# setup wait
wait = WebDriverWait(driver, 10)

URL = 'https://edtechimpact.com/categories/adaptive-learning'
driver.get(URL)

- Finish importing standard packages


In [25]:
page_source = BeautifulSoup(driver.page_source)
projects = page_source.find_all('div', class_ = 'ProductSearchPage__ProductListItem-a9rzyi-28 HVxyJ')
each_project_info = []
for project in projects:
    # project = projects[0]
    project_name = project.find('div', class_='ProductCard__HeaderName-sc-125gyi4-22 jKqvaJ').get_text()

    project_description = project.find('p', class_='ProductCard__HeaderDescription-sc-125gyi4-24 cnzABN').get_text()

    project_reviews_tag = project.find('div', class_='ProductCard__InfoItems-sc-125gyi4-6 keJxkX')
    see_all_reviews_id = project_reviews_tag.find('a').get('href')
    hash_position = see_all_reviews_id.index('#')
    see_all_reviews = 'https://edtechimpact.com' + see_all_reviews_id[:hash_position]
    each_project_info.append([project_name, project_description, see_all_reviews])

print(each_project_info) 

[['Eduten', 'An adaptive, AI-based learning platform co-designed and refined over the years with thousands of Finnish teachers.', 'https://edtechimpact.com/products/eduten-playground'], ['Reading Unlocked', 'Reading Unlocked teaches children to read confidently and fluently.', 'https://edtechimpact.com/products/reading-unlocked'], ['Boomwriter', 'Allows students to engage, collaborate, and improve their communication skills with classmates, and to develop their reading, writing, vocabulary and ...', 'https://edtechimpact.com/products/boomwriter'], ['Matific', 'A problem-solving approach to maths mastery, aligned to the primary maths curriculum in each country across the world.', 'https://edtechimpact.com/products/matific'], ['Blutick', 'AI-powered learning solution for the 11-16 Maths curriculum.', 'https://edtechimpact.com/products/blutick'], ['Numerise', 'Help your child master maths with Numerise.', 'https://edtechimpact.com/products/numerise'], ['Quizalize', 'Assign quizzes, worksh

In [106]:
# Scrape the data of 1 Linkedin profile, and write the data to a .CSV file
with open('adaptive_learning1.csv', 'w',  newline = '') as file_output:
    headers = [
        'project_name','project_description','projectURL',
        'company_name_check',
        'company_review_rate','what_is_company','recommended_usage',
        'Age Range', 'Subjects', 'Categories','Languages', 'Policies', 
        'Requirements', 'Training', 'Support', 'Pricing',
        'number of reviews', 'all_reviews'
    ]
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()

    for project_link in each_project_info:
        # driver.get(each_project_info[0][2])
        print(f'\n---Accessing {project_link[0]}')
        driver.get(project_link[2])
        sleep(5)

        content = driver.page_source.encode('utf-8').strip()
        company_page_source = BeautifulSoup(content, "html.parser")

        company_name_check = company_page_source.find('h1', class_='d-none d-md-block mb-0').get_text()
        print(company_name_check)

        company_review_rate = company_page_source.find('div', style='font-size: 1.2rem; margin-top: 3px;').get_text()

        what_is_company = company_page_source.find('div', class_='OurStory__StyledOurStory-nvq01x-12 gmQbeU').get_text()

        try:
            recommended_usage = company_page_source.find('div', style='font-style: italic;').get_text()
        except:
            recommended_usage = 'NA'

        print(company_review_rate)
        print(what_is_company)
        print(recommended_usage)

        details_tag = company_page_source.find('section', class_='ProductTrialDetails__StyledProductTrialDetails-sc-1a18mdj-1 ePcBDA')
        details_row = details_tag.find_all('div', class_='row')

        # 1st trial: this approach is quite manual
        # age = details_row[0].find('div', class_='col').get_text()
        # subject = details_row[1].find('div', class_='col').get_text()
        # categories = details_row[2].find('div', class_='col').get_text()
        # policies = details_row[3].find('div', class_='col').get_text()
        # requirements = details_row[4].find('div', class_='col').get_text()
        # training = details_row[5].find('div', class_='col').get_text()
        # support = details_row[6].find('div', class_='col').get_text()
        # pricing = details_row[-1].find('div', class_='col').get_text()
        # print(f"{age},\n {subject},\n {categories},\n {policies},\n {requirements},\n {training},\n {support},\n {pricing}")

        ls = [
            'Age Range', 'Subjects', 'Categories','Languages', 'Policies', 
            'Requirements', 'Training', 'Support', 'Pricing']

        d = {}
        # 3rd trial: this works, if the subject aren't available, the value is ''
        for title in ls:
            d[title] = ''

        # 2nd trial: this approach will get keyerror when write the output to csv as some pages don't include all the subtitle
        for information in details_row:
            var_name = information.find('div', class_='ProductTrialDetails__Title-sc-1a18mdj-3 ieAPui col-sm-6 col-md-3').get_text()
            value = information.find('div', class_='col').get_text()
            d[var_name] = value

        review_section = company_page_source.find_all('section')[-1]
        review_list = review_section.find_all('div', class_='Review-fotosa-8 fvYQhK')
        all_reviews = []
        for review in review_list:
            detailed_review = review.find('p', class_='Review__TheReview-fotosa-18 dseHFl').get_text()
            all_reviews.append(detailed_review)

        print(all_reviews)
        number_of_reviews = len(all_reviews)
        writer.writerow({
            headers[0]:project_link[0],
            headers[1]:project_link[1],
            headers[2]:project_link[2],
            headers[3]:company_name_check,
            headers[4]:company_review_rate,
            headers[5]:what_is_company,
            headers[6]:recommended_usage,
            headers[7]:d[headers[7]],
            headers[8]:d[headers[8]],
            headers[9]:d[headers[9]],
            headers[10]:d[headers[10]],
            headers[11]:d[headers[11]],
            headers[12]:d[headers[12]],
            headers[13]:d[headers[13]],
            headers[14]:d[headers[14]],
            headers[15]:d[headers[15]],
            headers[16]:number_of_reviews,
            headers[17]:all_reviews
            })

print('Mission Completed!')

 across the world. Matific produces award-winning interactive online mini-games, or episodes, to reinforce the mathematical concepts taught by primary school teachers. Matific’s mission is to proliferate high-quality maths education worldwide, empower teachers and parents, make maths intriguing, and open the gates to scientific thinking. Matific has hundreds of thousands of users worldwide. For more information, visit Matific.com. Read morePerformanceimprove attainment75%build student knowledge75%improve teaching efficiency50%reduce teacher workload50%Recommended UsageStudents who engage with Matific for 20 minutes a day have been proven to improve their mathematical understanding by 34 per cent.. 
Students who engage with Matific for 20 minutes a day have been proven to improve their mathematical understanding by 34 per cent.. 
['The focus on problem solving is excellent and children can guide their own learning']

---Accessing Blutick
Blutick
5/5(1 review)
What is Blutick?What is Blu